In [1]:
# pip install shapely

In [2]:
import pandas as pd
import requests
from shapely.geometry import MultiLineString
import json
import os
import io

# Source = "https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/reseau-cyclable-et-vert/" (site metropole)
url = "https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/reseau-cyclable-et-vert/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
response = requests.get(url)
print(response)

os.makedirs("inputs", exist_ok=True)
df = pd.read_csv(io.StringIO(response.text), sep=";")
df.to_csv("inputs/reseau-cyclable-et-vert.csv", index=False, sep=";")

<Response [200]>


In [3]:
df = pd.read_csv("inputs/reseau-cyclable-et-vert.csv", delimiter=";")
df.head(3)

,Geo Point,Geo Shape,code_insee,commune,pole,existant,hierarchisation,type_reseau,type_amenagement,longueur_m,annee_creation,annee_refection,num_rev
0,"43.72550137920509, 1.439074950444212","{""coordinates"": [[[1.43726908468553, 43.724177...",31230,Gratentour,Nord,oui,3,RC,piste cyclable,428,0,0,NaN
1,"43.727699067186926, 1.4416669876468986","{""coordinates"": [[[1.44066837417473, 43.727076...",31230,Gratentour,Nord,oui,3,RC,piste cyclable,250,0,0,NaN
2,"43.72026321352219, 1.423681885343167","{""coordinates"": [[[1.424836708339389, 43.72166...",31230,Gratentour,Nord,oui,4,RV,reseau vert,366,0,0,NaN


In [4]:
df["Geo Shape"] = df["Geo Shape"].apply(json.loads)
# print(df["WKT"][0]["coordinates"])

def geojson_to_wkt(geojson):
        lines = []
        for line in geojson["coordinates"]:
            lines.append(", ".join(f"{x[0]} {x[1]}" for x in line))
        return f"MULTILINESTRING(({'),('.join(lines)}))"

# Appliquer la conversion sur la colonne GeoJSON
df["WKT"] = df["Geo Shape"].apply(geojson_to_wkt)

print(df["WKT"].dtypes, df["WKT"])

object 0       MULTILINESTRING((1.43726908468553 43.724177057...
1       MULTILINESTRING((1.44066837417473 43.727076381...
2       MULTILINESTRING((1.424836708339389 43.72166874...
3       MULTILINESTRING((1.410035540831517 43.71647140...
4       MULTILINESTRING((1.404017189871906 43.72147414...
                              ...                        
4588    MULTILINESTRING((1.423536408947544 43.55524729...
4589    MULTILINESTRING((1.420836790422874 43.64598813...
4590    MULTILINESTRING((1.372341628442786 43.65663671...
4591    MULTILINESTRING((1.373400967401418 43.64334872...
4592    MULTILINESTRING((1.37858259020203 43.647586612...
Name: WKT, Length: 4593, dtype: object


In [5]:
df = df.drop(["Geo Shape"], axis=1)

os.makedirs("outputs", exist_ok=True)
df.to_csv("outputs/reseau-cyclable-et-vert_WKT.csv", index=False)

df.head(5)

,Geo Point,code_insee,commune,pole,existant,hierarchisation,type_reseau,type_amenagement,longueur_m,annee_creation,annee_refection,num_rev,WKT
0,"43.72550137920509, 1.439074950444212",31230,Gratentour,Nord,oui,3,RC,piste cyclable,428,0,0,NaN,MULTILINESTRING((1.43726908468553 43.724177057...
1,"43.727699067186926, 1.4416669876468986",31230,Gratentour,Nord,oui,3,RC,piste cyclable,250,0,0,NaN,MULTILINESTRING((1.44066837417473 43.727076381...
2,"43.72026321352219, 1.423681885343167",31230,Gratentour,Nord,oui,4,RV,reseau vert,366,0,0,NaN,MULTILINESTRING((1.424836708339389 43.72166874...
3,"43.71856170503369, 1.4064929954037269",31091,Bruguières,Nord,non,1,RC,NaN,838,0,0,REV 7 Nord,MULTILINESTRING((1.410035540831517 43.71647140...
4,"43.72235264766178, 1.4050290020519032",31091,Bruguières,Nord,non,1,RC,NaN,275,0,0,REV 7 Nord,MULTILINESTRING((1.404017189871906 43.72147414...
